In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf
from scipy.io import loadmat, savemat
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import scipy.io as sio
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


dense_layers = [0, 1, 2, 3]
dense_sizes = [32, 64, 128, 256, 512]
conv_layers = [1, 2, 3, 4]
num_filters = [32, 64, 128, 256]
kernel_sizes = [2000, 2250, 2500, 2750, 3000]

In [29]:
basePath = '8_antenna_100_snapshot_1000sample_minus20_20_snr'

rootPath = os.getcwd()
print(rootPath)
# Windoes path
filePath = 'c:\\Users\\HYPC300\\OneDrive - 한양대학교\\GitHub\\Deeplearning-Autoencoder-DOA\\data';
#  MAC path:
# filePath = '/Users/button/Deeplearning-Autoencoder-DOA/data'
savePath = filePath + '\\result' +'\\' + basePath

if not os.path.isdir(savePath):
    os.makedirs(savePath)
print(savePath)

c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\[2]Tensorflow\[1]DAE_DOA_implementation
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr


In [30]:
# Input train noisy dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath+'/*/train/train_data.*', shuffle=False)
# for i in dataset.as_numpy_iterator():
#     print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['train_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 

data_train = data_set[0]
for i in data_set:
    data_train = np.vstack((data_train,i)) # stack all the datasets vertically
    
data_train = data_train[5000:] # the first dataset is included in twice
data_train.shape

(205000, 64)

In [31]:
# Input train noiseless dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath + '/*/train/train_origin_data.*', shuffle=False)
# for i in dataset.as_numpy_iterator():
#     print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['train_origin_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 

data_origin = data_set[0]
for i in data_set:
    data_origin = np.vstack((data_origin,i)) # stack all the datasets vertically
    
data_origin = data_origin[5000:] # the first dataset is included in twice
data_origin.shape

(205000, 64)

In [32]:
# train data shuffle
np.random.seed(2020)
index = np.arange(data_train.shape[0])
print(index)
np.random.shuffle(index)
data_train = data_train[index]
data_origin = data_origin[index]

[     0      1      2 ... 204997 204998 204999]


In [38]:
# Model configuration
nSample = x_train.shape[0]
epoch = 100
batch_size = 64
validation_split = 0.2

save_dir = os.path.join(savePath, 'save_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
print(save_dir)

c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr\save_models


In [35]:
class DOA_Denoising(Model):
    def __init__(self):
        super(DOA_Denoising, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(32, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(16, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(8, activation='relu')]) # * 300 200 100
        
        self.decoder = tf.keras.Sequential([
            layers.Dense(16, activation='relu'), #* 200 300 400
            # layers.Dropout(0.2),
            layers.Dense(32, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(64, activation='linear')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
autoencoder = DOA_Denoising() 

In [193]:
# %reload_ext tensorboard
# %tensorboard --logdir logs

In [40]:
model_name = (
    f"epoch-{epoch}-batch-{batch_size}-dense-300-200-100-time-{int(time.time())}"
)
print(model_name)

file_path = os.path.join(save_dir, model_name)
print(file_path)

# callbacks_list
callbacks_list = [
    EarlyStopping(monitor="val_loss", mode="min", verbose=0, patience=50),
    ModelCheckpoint(
        filepath=file_path,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="min",
    ),
    # TensorBoard(
    #     log_dir="./logs/{}".format(model_name),
    #     histogram_freq=1,
    #     write_graph=False,
    #     write_images=False,
    # ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=10,
        verbose=0,
        mode="min",
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
    ),
]

autoencoder.compile(optimizer="adam", loss="mse")

epoch-100-batch-64-dense-300-200-100-time-1645196416
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr\save_models\epoch-100-batch-64-dense-300-200-100-time-1645196416


In [41]:
history = autoencoder.fit(
    # dataset_final.take(num_train),
    data_train,
    data_origin,
    epochs=epoch,
    batch_size=batch_size,
    shuffle=True,
    # validation_data = dataset_final.skip(205000*0.8),
    validation_split=validation_split,
    callbacks=callbacks_list,  #! Do not put [] around callbacks_list variable
)

autoencoder.encoder.summary()
autoencoder.decoder.summary()

Epoch 1/100
2563/2563 [==============================] - 2s 843us/step - loss: 0.0910 - val_loss: 0.1024
Epoch 2/100
2563/2563 [==============================] - 2s 921us/step - loss: 0.0905 - val_loss: 0.1027
Epoch 3/100
2563/2563 [==============================] - 2s 955us/step - loss: 0.0906 - val_loss: 0.1041
Epoch 4/100
2563/2563 [==============================] - 2s 824us/step - loss: 0.0906 - val_loss: 0.1068
Epoch 5/100
2563/2563 [==============================] - 2s 819us/step - loss: 0.0904 - val_loss: 0.1023
Epoch 6/100
2563/2563 [==============================] - 2s 812us/step - loss: 0.0904 - val_loss: 0.1036
Epoch 7/100
2563/2563 [==============================] - 2s 779us/step - loss: 0.0903 - val_loss: 0.1021
Epoch 8/100
2563/2563 [==============================] - 2s 922us/step - loss: 0.0901 - val_loss: 0.1013
Epoch 9/100
2563/2563 [==============================] - 2s 827us/step - loss: 0.0900 - val_loss: 0.1043
Epoch 10/100
2563/2563 [==============================]

In [42]:
plt.plot(history.history["loss"], label="Traing Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
# plt.savefig('10.png')
# plt.savefig('SNR_-30_training_loss.png')

In [43]:
# Input test noisy dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath + '/20dB/test/test_data.*', shuffle=False)
for i in dataset.as_numpy_iterator():
    print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['test_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 
data_test = data_set[0]
for i in data_set:
    data_test = np.vstack((data_test,i)) # stack all the datasets vertically
data_test = data_test[1000:] # the first dataset is included in twice
data_test.shape
# export data for MATLAB processing
print(savePath)
savePath_ = savePath + '\\test_data.mat'
print(savePath_)
savemat(savePath_, {'test_data':data_test})

b'8_antenna_100_snapshot_1000sample_minus20_20_snr\\20dB\\test\\test_data.mat'
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr\test_data.mat


(1000, 8)
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr/denoised_data.mat


In [47]:
encoded_data = autoencoder.encoder(data_test).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()
print(encoded_data.shape)

# Save the denoised data for MATLAB processing
print(savePath)
savePath_ = savePath + '/denoised_data.mat'
print(savePath_)
savemat(savePath_, {'denoised_data':decoded_data})

ind = 32
plt.plot(data_test[ind],'b')
plt.plot(decoded_data[ind],'k')
plt.fill_between(np.arange(64),decoded_data[ind],data_test[ind],color='lightcoral')
plt.legend(["Input", "Reconstruction","Error"])
# plt.savefig('SNR_-30_autoencoder_compare.png')

In [296]:
# Input test label dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath + '/20dB/test/test_label.*', shuffle=False)
# for i in dataset.as_numpy_iterator():
#     print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['test_label']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 
data_label = data_set[0]
for i in data_set:
    data_label = np.vstack((data_label,i)) # stack all the datasets vertically
data_label = data_label[1000:] # the first dataset is included in twice
data_label.shape

print(savePath)
savePath_ = savePath + '\\test_label.mat'
print(savePath_)
savemat(savePath_, {'test_label':data_label})

c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\random_two source_10snapshots_1000sample
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\random_two source_10snapshots_1000sample\test_label.mat


In [62]:
x_test = sio.loadmat('vec')
test_max = x_test['vec']
test_max = test_max.reshape(1,64)
test_max.shape

(1, 64)

In [63]:

encoded_data = autoencoder.encoder(test_max).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()
print(encoded_data.shape)

# Save the denoised data for MATLAB processing
print(savePath)
savePath_ = savePath + '/denoised_data.mat'
print(savePath_)
savemat(savePath_, {'denoised_data':decoded_data})


(1, 8)
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr
c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\data\result\8_antenna_100_snapshot_1000sample_minus20_20_snr/denoised_data.mat
